In [1]:
import pandas as pd

df = pd.read_csv('data.csv')
df

label                                             review
0        0                          味道一般，酸辣土豆丝酸的没法吃。。。肝尖也做的一般
1        0  订的菜没有了，换了个便宜的菜，说了多退少补，结果没把钱给我退回来。诚信很有问题。锅包肉又硬又...
2        0                                       太慢,太慢,太慢,不好吃
3        0                                 商家距离近，但速度太慢，饼子都凉了。
4        0  久久鸭的口味偏颇，辣味不是很浓。总体而言还行。但是没有餐具，虽然我的单里说明要餐具了，这是一...
..     ...                                                ...
195      0                                      两个小时才送到,我是真苦了
196      1         晚了半小时左右吧……不过最近这么冷还是辛苦了，估计大家都不想出门吧。超级好吃！推荐！
197      0                        没吃过这么难吃的水煮牛肉，刚送来吃一口就扔了。有股膻味
198      0                                        送餐慢，肉少，味道不好
199      0                             下错单了,不过冷面味道不错,但是没有给我小菜

[200 rows x 2 columns]

## 小实验

In [27]:
%%time
from openai import OpenAI
from pydantic import BaseModel
from typing import List
import os
import instructor

#结构化输出
class Sentiment(BaseModel):
    senti_label: str
    senti_score: float


#Prompt提示
PROMPT_TEXT = "根据评论内容，返回文本的情感类别(pos、neg、neo)和对应的情感得分(取值范围0~1)"

#实验数据
COMMENT_CONTENT = '11点14订餐，13点20饭才到，2个小时才把我的午饭送到，而且还是打了2次客服电话，1次投诉电话才给送来，要是不打电话都不知道几点能吃上午饭？'


client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="NA",  # required, but unused
    ),
    mode = instructor.Mode.JSON,
)


resp = client.chat.completions.create(
    model = "qwen2.5:3b",
    messages=[
        {"role": "system", "content": PROMPT_TEXT}, #提示PROMP
        {"role": "user", "content": COMMENT_CONTENT} #评论文本
    ],
    response_model = Sentiment,
    max_retries = 3
)


print(resp.model_dump_json(indent=2))

{
  "senti_label": "neg",
  "senti_score": 0.1
}
CPU times: user 37.5 ms, sys: 4.47 ms, total: 42 ms
Wall time: 577 ms


## 同步代码

In [ ]:
%%time
import pandas as pd
from tqdm import tqdm
from openai import OpenAI
from pydantic import BaseModel
from typing import List
import numpy as np
import os
import instructor

 

#结构化输出
class Sentiment(BaseModel):
    senti_label: str
    senti_score: float
    

#Prompt提示
PROMPT_TEXT = "根据评论内容，返回文本的情感类别(pos、neg)和情感得分(取值范围 -1~1)" 

client = instructor.from_openai(
    OpenAI(
        base_url="http://localhost:11434/v1",
        api_key="NA",  # required, but unused
    ),
    mode = instructor.Mode.JSON,
)


labels = []
scores = []

#读取数据
df = pd.read_csv('data.csv')
for review in tqdm(df['review']):
    try:
        resp = client.chat.completions.create(
        model = 'qwen2.5:7b',  #选择模型。 3b、7b等
        messages=[
            {"role": "system", "content": PROMPT_TEXT},  #提示
            {"role": "user", "content": review}   #评论文本
        ],
        response_model = Sentiment,
        max_retries = 3  #最大(失败）的重试次数。 
    )
        
        labels.append(resp.senti_label)
        scores.append(resp.senti_score)
    except:
        labels.append('NA')
        scores.append(np.nan)
        
    
df['SentiLabel'] = labels
df['SentiScore'] = scores
#保存结果
df.to_csv('qwen2.5-7b-result.csv', index=False)
df

## 准确率计算方法

In [ ]:
expression = "(label == 1) & (SentiLabel == 'pos') | (label == 0) & (SentiLabel == 'neg')"
correct_ratio = len(df.query(expression))/ len(df)

print(f'准确率: {correct_ratio*100}%')

## 异步代码

In [ ]:
%%time

import pandas as pd
from tqdm.asyncio import tqdm_asyncio
# 使用AsyncOpenAI代替OpenAI以支持异步操作
from openai import AsyncOpenAI
from pydantic import BaseModel
from typing import List
import numpy as np
import os
import instructor
import asyncio

# 结构化输出
class Sentiment(BaseModel):
    senti_label: str
    senti_score: float

# Prompt提示
PROMPT_TEXT = "根据评论内容，返回文本的情感类别(pos、neg)和情感得分(取值范围 -1~1)"

client = instructor.from_openai(
    AsyncOpenAI(
        base_url="http://localhost:11434/v1",
        api_key="NA",  # required, but unused
    ),
    mode=instructor.Mode.JSON,
)

async def analyze_review(review):
    try:
        resp = await client.chat.completions.create(
            model='qwen2.5:3b',  # 选择模型。 3b、7b等
            messages=[
                {"role": "system", "content": PROMPT_TEXT},  # 提示
                {"role": "user", "content": review}  # 评论文本
            ],
            response_model=Sentiment,
            max_retries=3  # 最大(失败）的重试次数。
        )
        return resp.senti_label, resp.senti_score
    except Exception as e:
        print(f"Error processing review: {e}")
        return 'NA', np.nan

async def main():
    # 读取数据
    df = pd.read_csv('data.csv')
    tasks = [analyze_review(review) for review in df['review']]
    results = await tqdm_asyncio.gather(*tasks)
    
    labels, scores = zip(*results)
    df['SentiLabel'] = labels
    df['SentiScore'] = scores
    
    # 保存结果
    df.to_csv('async-qwen2.5-3b-result.csv', index=False)

# 检查是否已经在运行的事件循环中
try:
    asyncio.get_running_loop()
    # 如果在交互模式下运行，直接调度main()而不使用asyncio.run
    asyncio.create_task(main())
except RuntimeError:
    # 如果没有正在运行的事件循环，使用asyncio.run(main())
    asyncio.run(main())